# Quantum Teleportation with qoqo & the use of conditional measurements

This notebook is designed to demonstrate the use of conditional measurements, by following through an example of quantum state teleportation.

In quantum teleportation there are two end users: The first user, Alice, wishes to send a particular quantum state to the second user, Bob. The protocol requires a total of three qubits, and the transmission of two classical bits. 

The sender Alice controls qubits 0 and 1, and the reciever Bob controls qubit 2. 



In [17]:
from qoqo_quest import Backend
from qoqo import Circuit
from qoqo import operations as ops 
from math import pi


## State preparation

The first step is to prepare the quantum state which Alice will send to Bob. As an example, the most general single qubit quantum state is given by:
\begin{equation}
|\psi \rangle = cos(\frac{\theta}{2}) |0 \rangle + e^{i \phi} sin(\frac{\theta}{2}) |1 \rangle.
\end{equation}
This state can be prepared by a sequence of two single qubit rotations. In the code block below we first define a function that takes the angles $\theta$ and $\phi$ as input and prepares qubit 0 of a quantum register in the state $| \psi \rangle$.

Next we use an instance of the function with the angles $\theta=\frac{\pi}{2}$ and $\phi=0$ to create a circuit which prepares the state: 
\begin{equation}
|\psi \rangle = \frac{1}{\sqrt{2}} \big ( |0 \rangle + |1 \rangle \big ) = | + \rangle.
\end{equation}

In [18]:
def prep_psi(Theta: float, Phi: float) -> Circuit:
    circuit = Circuit()
    circuit += ops.RotateY(qubit=0, theta=Theta)
    circuit += ops.RotateZ(qubit=0, theta=Phi)
    return circuit

init_circuit = prep_psi(pi/2, 0.0)


## Preparing an entangled resource state

Quantum teleportation requires that the end users initially share an entangled resource state, 
\begin{equation}
|\Phi_{+} \rangle = \frac{1}{\sqrt(2)} \big ( |00 \rangle + |11 \rangle \big ) .
\end{equation}

The following circuit prepares the state $|\Phi_{+} \rangle$ between qubit 1, held by Alice, and qubit 2, held by Bob.

In [19]:
entangling_circ = Circuit()
entangling_circ += ops.Hadamard(qubit=1)
entangling_circ += ops.CNOT(control=1, target=2)

## Encoding the state to be sent in the entangled resource state

The next step of the procedure is to encode the state of qubit 0, $\psi$, into the entangled resource state. This is accomplished by way of the circuit defined below, which is similar to that used to prepare the entangled resource. 


In [20]:
encoding_circ = Circuit()
encoding_circ += ops.CNOT(control=0, target=1)
encoding_circ += ops.Hadamard(qubit=0)

## State transfer part 1: Measurement

At this stage in the process both of Alice's qubits, 0 and 1, are measured. The measurement consumes the entangled resource and leaves the state of qubit 2,Bob's qubit, in a state that depends on the two measurement outcomes. 

Let us call the classical bit which results from measuring qubit 0 'M1' and the bit resulting from measuring qubit 1 'M2'. The circuit below defines the classical register named 'M1M2', performs the measurement of qubits 0 and 1, and stores the results in the register 'M1M2'. 

In [21]:
meas_circ = Circuit()
meas_circ += ops.DefinitionBit(name='M1M2', length=2, is_output=True) #for classical bits corresponding to measurement outcomes
meas_circ += ops.MeasureQubit(qubit=0,readout='M1M2',readout_index=0)
meas_circ += ops.MeasureQubit(qubit=1,readout='M1M2',readout_index=1)

## Defining the circuit for a conditional operation

Conditional operations in qoqo have three inputs: the name of a classical register containing boolean values, the index of the register containing the value to be used to condition the operation, and the operation or sequence of operations to be performed if the boolean condition value is True. 

To prepare the third input, it is necessary to create circuit snippets corresponding to the operations to be completed if the condition is True. 

In the case of quantum teleportation, we need two conditional operations. The first is a Pauli Z acting on Bob's qubit, conditioned on the measurement result M1. The second is a Pauli X acting on Bob's qubit, conditioned on the measurement result M2. 

Hence we prepare circuit snippets correspponding to a Pauli Z and a Pauli X operation.

In [22]:
conditional_Z = Circuit()
conditional_Z += ops.PauliZ(qubit=2)

conditional_X = Circuit()
conditional_X += ops.PauliX(qubit=2)


## State transfer part 2: conditional operations

The final stage of the teleportation protocol is to perform corrections to the state of Bob's qubit 2, according to the measurement outcomes 'M1' and 'M2'.

The below circuit makes use of the circuit snippets defined above to perform the conditional corrections to the state of qubit 2. 2. 

In [23]:
conditional_circ = Circuit()
conditional_circ += ops.PragmaConditional(condition_register='M1M2',condition_index=1, circuit=conditional_X)
conditional_circ += ops.PragmaConditional(condition_register='M1M2',condition_index=0, circuit=conditional_Z)

## Putting it all together

Combining each of the circuits we have defined yeilds the full teleportation protocol. We can verify that the protocol is successful by reading out the final state vector and comparing it to the state which was to be sent, $|\psi \rangle$. 

In [24]:
verification = Circuit()
# Create register for state vector readout
verification += ops.DefinitionComplex(name='psi', length=8, is_output=True) 
verification += ops.PragmaGetStateVector(readout='psi', circuit=Circuit())

# Combine parts for full protocol
teleportation_circuit = init_circuit + entangling_circ + encoding_circ + meas_circ + conditional_circ + verification

# Run simulation and collect outputs
backend = Backend(number_qubits=3)
(result_bit_registers, result_float_registers, result_complex_registers)=backend.run_circuit(teleportation_circuit)

# View measurement outcomes and post-protocol state of qubits
print(result_bit_registers['M1M2'])
print(result_complex_registers['psi'])

[[True, False]]
[[0j, (0.7071067811865476+0j), 0j, 0j, (-0+0j), (0.7071067811865475-0j), (-0+0j), (-0+0j)]]
